In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/train.csv')
df.head()

,bant_submit,customer_country,business_unit,com_reg_ver_win_rate,customer_idx,customer_type,enterprise,historical_existing_cnt,id_strategic_ver,it_strategic_ver,...,response_corporate,expected_timeline,ver_cus,ver_pro,ver_win_rate_x,ver_win_ratio_per_bu,business_area,business_subarea,lead_owner,is_converted
0,1.0,/Quezon City/Philippines,AS,0.066667,32160,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Engineering,0,True
1,1.0,/PH-00/Philippines,AS,0.066667,23122,End-Customer,Enterprise,12.0,NaN,NaN,...,LGEPH,less than 3 months,1,0,0.003079,0.026846,corporate / office,Advertising,1,True
2,1.0,/Kolkata /India,AS,0.088889,1755,End-Customer,Enterprise,144.0,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,Construction,2,True
3,1.0,/Bhubaneswar/India,AS,0.088889,4919,End-Customer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,1,0,0.003079,0.026846,corporate / office,IT/Software,3,True
4,1.0,/Hyderabad/India,AS,0.088889,17126,Specifier/ Influencer,Enterprise,NaN,NaN,NaN,...,LGEIL,less than 3 months,0,0,0.003079,0.026846,corporate / office,NaN,4,True


In [3]:
df.isnull().sum()

bant_submit                    0
customer_country             982
business_unit                  0
com_reg_ver_win_rate       44731
customer_idx                   0
customer_type              43961
enterprise                     0
historical_existing_cnt    45543
id_strategic_ver           55855
it_strategic_ver           58178
idit_strategic_ver         54734
customer_job               18733
lead_desc_length               0
inquiry_type                 941
product_category           19374
product_subcategory        50064
product_modelname          50070
customer_country.1           982
customer_position              0
response_corporate             0
expected_timeline          30863
ver_cus                        0
ver_pro                        0
ver_win_rate_x             40882
ver_win_ratio_per_bu       43995
business_area              40882
business_subarea           53773
lead_owner                     0
is_converted                   0
dtype: int64

### 문자형 변수 처리
+ customer_country, business_unit, customer_type, enterprise, customer_job, inquiry_type, product_category,<br/> product_subcategory, product_modelname, customer_position, response_corporate, expected_timeline, <br/> business_area, business_subarea

In [108]:
df['business_unit'].unique() # 라벨인코딩

array(['AS', 'ID', 'IT', 'Solution', 'CM'], dtype=object)

In [4]:
df['customer_type'].unique()

array(['End-Customer', 'Specifier/ Influencer', 'Service Partner',
       'Channel Partner', nan, 'Corporate', 'End Customer',
       'Specifier / Influencer', 'Solution Eco-Partner', 'Distributor',
       'Other', 'System Integrator', 'Consultant', 'Installer',
       'Homeowner', 'Others', 'Technician', 'Installer/Contractor',
       'Engineer', 'Manager / Director', 'Developer', 'Etc.',
       'Architect/Consultant', 'End-user', 'HVAC Engineer', 'Reseller',
       'Software/Solution Provider', 'Software / Solution Provider',
       'Dealer/Distributor', 'Technical Assistant', 'Commercial end-user',
       'Interior Designer', 'Home Owner', 'Administrator'], dtype=object)

In [11]:
df['customer_type'].value_counts()

customer_type
end_user                10651
influencer               2568
channel partner          1368
service partner           349
solution eco-partner      146
etc                        67
installer/contractor       52
corporate                  31
hvac engineer              23
engineer                   20
developer                  17
technician                 16
consultant                 15
home_owner                 15
Name: count, dtype: int64

In [6]:
df['customer_type'] = df['customer_type'].str.lower()
df['customer_type'] = df['customer_type'].replace(['etc.', 'other', 'others'], 'etc')
df['customer_type'] = df['customer_type'].replace(['end-customer', 'end customer', 'end-user'], 'end_user')
df['customer_type'] = df['customer_type'].replace(['specifier/ influencer', 'specifier / influencer'], 'influencer')
df['customer_type'] = df['customer_type'].replace(['homeowner', 'home owner'], 'home_owner')
df['customer_type'] = df['customer_type'].replace(['software/solution provider', 'software / solution provider'], 'solution_provider')

In [8]:
# 값의 빈도 계산
value_counts = df['customer_type'].value_counts()

# 100개 미만인 값들을 식별
values_to_replace = value_counts[value_counts < 10].index

# 해당 값들을 'etc'로 대체
df['customer_type'] = df['customer_type'].apply(lambda x: 'etc' if x in values_to_replace else x)

In [14]:
df['customer_type'].unique() # 라벨인코딩 14개

array(['end_user', 'influencer', 'service partner', 'channel partner',
       nan, 'corporate', 'solution eco-partner', 'etc', 'consultant',
       'home_owner', 'technician', 'installer/contractor', 'engineer',
       'developer', 'hvac engineer'], dtype=object)

In [15]:
df['enterprise'].unique() # 0, 1로 mapping

array(['Enterprise', 'SMB'], dtype=object)

In [20]:
df['inquiry_type'].value_counts()

inquiry_type
quotation_or_purchase_consultation    42379
sales inquiry                          9981
usage or technical consultation        1668
etc                                    1553
product information                    1237
trainings                               434
services                                415
request for partnership                 297
request a demo                          184
technical support                       110
sales                                   100
Name: count, dtype: int64

In [17]:
df['inquiry_type'] = df['inquiry_type'].str.lower()
# 같은 의미의 다른 문자열들을 통일
df['inquiry_type'] = df['inquiry_type'].replace(['etc.', 'other', 'other_', 'others'], 'etc')
df['inquiry_type'] = df['inquiry_type'].replace(['quotation or purchase consultation', 'request for quotation or purchase', 'quotation_or_purchase_consultation', 'purchase or quotation', 'quotation_'], 'quotation_or_purchase_consultation')
df['inquiry_type'] = df['inquiry_type'].replace(['usage or technical consultation', 'technical consultation', 'request for technical consulting', 'usage_or_technical_consultation', 'technical_consultation', 'technical'], 'usage or technical consultation')

In [19]:
# 값의 빈도 계산
value_counts = df['inquiry_type'].value_counts()

# 100개 미만인 값들을 식별
values_to_replace = value_counts[value_counts < 100].index

# 해당 값들을 'etc'로 대체
df['inquiry_type'] = df['inquiry_type'].apply(lambda x: 'etc' if x in values_to_replace else x)

In [21]:
df['inquiry_type'].unique() # 라벨인코딩

array(['quotation_or_purchase_consultation', 'product information', 'etc',
       'usage or technical consultation', 'trainings', 'services',
       'sales inquiry', 'technical support', 'request for partnership',
       nan, 'sales', 'request a demo'], dtype=object)

In [25]:
df['product_category'].value_counts()

product_category
interactive digital board      6154
vrf                            5640
etc.                           4098
multi-split                    3674
video wall signage             2029
led signage                    1895
interactive signage            1862
single-split                   1569
rac                            1447
oled signage                   1350
hotel tv                       1112
chiller                        1051
standard signage                996
medical display                 962
lg one:quick series             768
monitor                         762
one:quick series                668
heating                         655
high brightness signage         477
ventilation                     420
teto ou cassete inverter        308
control                         263
multi inverter                  257
ar condicionado residencial     224
high brightness                 219
software solution               191
accessories                     179
special sig

In [24]:
# 값의 빈도 계산
value_counts = df['product_category'].value_counts()

# 100개 미만인 값들을 식별
values_to_replace = value_counts[value_counts < 100].index

# 해당 값들을 'Other'로 대체
df['product_category'] = df['product_category'].apply(lambda x: 'etc.' if x in values_to_replace else x)

In [27]:
df['product_category'].unique()

array(['multi-split', 'single-split', 'vrf', 'chiller', 'etc.', 'rac',
       'teto ou cassete inverter', nan, 'software solution',
       'ventilation', 'heating', 'multi inverter', 'standard signage',
       'high brightness signage', 'interactive signage',
       'video wall signage', 'led signage', 'hotel tv', 'oled signage',
       'one:quick series', 'special signage', 'accessories',
       'interactive digital board', 'lg one:quick series', 'pro:centric',
       'webos', 'high brightness', 'hospital tv', 'monitor', 'pc',
       'medical display', 'control', 'ar condicionado residencial'],
      dtype=object)

In [28]:
df.drop(columns=['product_subcategory', 'product_modelname'], inplace = True)

In [33]:
df['customer_position'].value_counts().head(30)

customer_position
etc                  27626
manager               8217
ceo/founder           7984
director              4843
associate/analyst     2597
partner               2536
entry level           2216
c-level executive      851
trainee                849
vice president         509
intern                 446
end-user               424
hospital               201
Name: count, dtype: int64

In [31]:
# 값의 빈도 계산
value_counts = df['customer_position'].value_counts()
values_to_replace = value_counts[value_counts < 100].index
df['customer_position'] = df['customer_position'].apply(lambda x: 'etc' if x in values_to_replace else x)

In [32]:
df['customer_position'] = df['customer_position'].replace(['none', 'other', 'others'], 'etc')

In [34]:
df['customer_position'].unique()

array(['entry level', 'ceo/founder', 'partner', 'manager',
       'vice president', 'associate/analyst', 'c-level executive', 'etc',
       'director', 'intern', 'trainee', 'hospital', 'end-user'],
      dtype=object)

In [35]:
df['response_corporate'].unique()

array(['LGEPH', 'LGEIL', 'LGEAF', 'LGESJ', 'LGESL', 'LGESP', 'LGEGF',
       'LGESA', 'LGEUS', 'LGECB', 'LGEMS', 'LGEEG', 'LGEEF', 'LGEAP',
       'LGEIN', 'LGEUK', 'LGEKR', 'LGEPS', 'LGECI', 'LGECL', 'LGETK',
       'LGELF', 'LGEPT', 'LGEPR', 'LGEDG', 'LGERO', 'LGEMK', 'LGEPL',
       'LGECZ', 'LGEES', 'LGEAR', 'LGEHK', 'LGEML', 'LGEJP', 'LGEHS',
       'LGEAS', 'LGEYK', 'LGEIS', 'LGEBN', 'LGEFS', 'LGESW', 'LGEMC',
       'LGEAG', 'LGEEB', 'LGETH', 'LGEVH', 'LGECH', 'LGELA', 'LGETT',
       'LGERA', 'LGEUR', 'LGEIR', 'LGEBT'], dtype=object)

In [220]:
df['expected_timeline'].unique() # 컬럼 삭제

array(['less than 3 months', nan, '3 months ~ 6 months',
       '9 months ~ 1 year', 'more than a year', '6 months ~ 9 months',
       'quote has been sent to customer.',
       'partner was exploring on eol model for some requiremeent. no designated sales or closure expected hence closing in the system.',
       'already close in july purchse with rd',
       'client not interested in product.. receing call and not answering properly',
       'being followed up', 'being followed up.',
       'repeated inquiry from client. he was just inquiring but have not confimred on purchase. we have tried mulitple times but client is not interested in buying.',
       'update- 7th aug--demo given. customer will confirm next week',
       "tried to reach him multipe times from last 7 days but couldn't get through",
       'details send', 'discused with ankit in dehli',
       'requires detail for tender. no purchase requirement right now.',
       'partner is into static signages. he need to connec

In [36]:
df['ver_win_rate_x'].fillna(0, inplace = True)

In [37]:
df['ver_win_ratio_per_bu'].fillna(0, inplace = True)

In [233]:
df['business_area'].value_counts() # 그대로 레이블인코딩, # 열 삭제가 낫겠다

business_area
corporate / office                4097
retail                            3011
hotel & accommodation             2013
special purpose                   1929
residential (home)                1818
education                         1727
hospital & health care            1199
factory                           1035
government department              650
public facility                    519
transportation                     306
power plant / renewable energy     113
Name: count, dtype: int64

In [242]:
df['lead_owner'].value_counts().head(30) # 열 삭제

lead_owner
174    4627
175    4120
97     2499
23     2453
194    2208
42     1215
96      981
185     880
363     839
115     734
38      693
125     690
402     490
108     487
124     468
816     464
341     462
167     459
100     457
0       429
68      427
16      413
487     405
176     374
40      355
342     346
662     327
422     326
27      317
359     305
Name: count, dtype: int64

In [249]:
df['business_area'].value_counts().head(30)

business_area
corporate / office                4097
retail                            3011
hotel & accommodation             2013
special purpose                   1929
residential (home)                1818
education                         1727
hospital & health care            1199
factory                           1035
government department              650
public facility                    519
transportation                     306
power plant / renewable energy     113
Name: count, dtype: int64

In [248]:
(df.groupby('business_area')['is_converted'].sum() / df.groupby('business_area')['is_converted'].count()).sort_values(ascending=False)

business_area
hospital & health care            0.311093
power plant / renewable energy    0.176991
factory                           0.085990
public facility                   0.065511
corporate / office                0.065414
education                         0.063115
government department             0.060000
hotel & accommodation             0.059613
transportation                    0.058824
retail                            0.052474
special purpose                   0.052359
residential (home)                0.024752
Name: is_converted, dtype: float64

In [247]:
df.groupby('lead_owner')['is_converted'].sum().sort_values(ascending=False) # 삭제하자

lead_owner
437     135
487     117
831     104
480      98
375      88
       ... 
592       0
79        0
80        0
584       0
1114      0
Name: is_converted, Length: 984, dtype: int64

In [251]:
df['business_unit'].value_counts() # 라벨인코딩

business_unit
ID          25563
AS          24774
IT           8664
Solution      296
CM              2
Name: count, dtype: int64

In [38]:
df['com_reg_ver_win_rate'].fillna(0, inplace = True)

In [255]:
df['customer_idx'].value_counts()

customer_idx
47466    7686
25096    2421
37680     367
18030     331
21321     162
         ... 
39487       1
41241       1
39422       1
22515       1
30268       1
Name: count, Length: 35112, dtype: int64

In [258]:
df['customer_idx'].value_counts()

customer_idx
47466    7686
25096    2421
37680     367
18030     331
21321     162
         ... 
39487       1
41241       1
39422       1
22515       1
30268       1
Name: count, Length: 35112, dtype: int64

In [263]:
df['customer_idx'].value_counts().head(30)

customer_idx
47466    7686
25096    2421
37680     367
18030     331
21321     162
33350     135
43395      91
25309      90
33763      88
32240      84
37654      76
31864      71
37657      67
19804      65
29370      61
40344      59
21389      51
7195       50
37399      45
42067      43
14746      43
33776      33
742        30
6555       29
1747       28
46271      27
21883      26
37306      26
40491      26
4936       26
Name: count, dtype: int64

In [286]:
df['is_converted'].value_counts()

is_converted
False    54449
True      4850
Name: count, dtype: int64

In [285]:
df[df['customer_idx'] == 42067]['is_converted'].value_counts() # 25096 은 True만 존재 -> 25096은 빼고 학습하는게 어떨까 -> test 에는 25096이 없음. 컬럼 삭제

is_converted
False    43
Name: count, dtype: int64

In [39]:
# customer_country, customer_country.1 이 같음 하나 삭제
df.drop(columns=['customer_country.1'], inplace= True)
df[['it_strategic_ver', 'id_strategic_ver', 'idit_strategic_ver']] = df[['it_strategic_ver', 'id_strategic_ver', 'idit_strategic_ver']].fillna(0)

In [40]:
df['historical_existing_cnt'].fillna(0, inplace = True)

In [53]:
df['customer_job'].value_counts().head(50) # 레이블인코딩

customer_job
etc                               7129
engineering                       6342
administrative                    3359
education                         2287
sales                             2218
operations                        1961
purchasing                        1938
information technology            1937
business development              1801
consulting                        1349
arts and design                   1276
entrepreneurship                  1192
marketing                         1161
program and project management     948
media and communication            709
healthcare services                632
product management                 573
finance                            463
accounting                         396
support                            358
graphic/color art                  331
real estate                        295
film production                    241
3d/vfx art                         235
clinical specialist                227
radiology pr

In [52]:
df['customer_job'] = df['customer_job'].replace(['other', 'others'], 'etc')
# 값의 빈도 계산
value_counts = df['customer_job'].value_counts()
values_to_replace = value_counts[value_counts < 100].index
df['customer_job'] = df['customer_job'].apply(lambda x: 'etc' if x in values_to_replace else x)

In [58]:
df['customer_job'].unique() # 레이블인코딩 33

array(['purchasing', 'media and communication', 'engineering',
       'entrepreneurship', 'consulting', 'program and project management',
       'sales', 'etc', 'operations', nan, 'administrative',
       'business development', 'information technology', 'accounting',
       'education', 'healthcare services', 'human resources', 'support',
       'finance', 'marketing', 'legal', 'arts and design', 'real estate',
       'research', 'product management', 'community and social services',
       'radiology professional', 'graphic/color art',
       'medical imaging specialist', 'medical solution provider',
       '3d/vfx art', 'film production', 'surgery professional',
       'clinical specialist'], dtype=object)

In [54]:
df['lead_desc_length'].value_counts()

lead_desc_length
3      10378
14      3180
2        934
10       891
4        884
       ...  
824        1
559        1
676        1
907        1
726        1
Name: count, Length: 909, dtype: int64

In [59]:
df['customer_country'] = df['customer_country'].str.lower()
df['country'] = df['customer_country'].apply(lambda x : str(x).split('/')[-1])

In [327]:
df['country'].to_csv('country.csv')

In [60]:
country = df['customer_country'].drop_duplicates()
country

0        /quezon city/philippines
1              /ph-00/philippines
2                 /kolkata /india
3              /bhubaneswar/india
4                /hyderabad/india
                   ...           
59292         /baturaja/indonesia
59293             /sląskie/poland
59295        /bogotá dc /colombia
59296                 /pisco/peru
59297    /santa cruz bolivia/peru
Name: customer_country, Length: 13712, dtype: object

In [68]:
country = df['customer_country'].apply(lambda x : str(x).split('/')[-1])
country = country.replace('.*united states.*', 'usa', regex = True)
country = country.replace('.*usa.*', 'usa', regex = True)

In [69]:
country.unique() # 삭제

array(['philippines', 'india', 'nigeria', 'saudi arabia', 'singapore',
       'brazil', 'u.a.e', 'south africa', 'usa', 'colombia', 'mexico',
       'ghana', 'egypt', 'democratic republic of the congo', 'ethiopia',
       'australia', 'nan', 'kenya', 'indonesia', 'oman', 'pakistan',
       'united kingdom', 'guatemala', 'panama', 'canada', 'bangladesh',
       'papua new guinea', 'united republic of tanzania', 'qatar',
       'afghanistan', 'chile', 'mozambique', 'turkey', 'türkiye',
       'el salvador', 'togo', 'jordan', 'iraq', 'israel', 'sri lanka',
       'south korea', 'portugal', 'mauritania', 'uruguay', 'peru',
       'germany', 'romania', 'norway', 'jamaica', '', 'hungary', 'poland',
       'czech', 'spain', 'argentina', 'ecuador', 'senegal', 'hong kong',
       'malaysia', 'japan', 'kuwait', 'ireland', 'albania', 'greece',
       'barranquilla', 'algeria', 'nicaragua', 'slovenia', 'italy',
       'netherlands', 'dominican republic', 'france',
       'abdohabrouk@gmail.com', '

In [330]:
df['customer_country'].apply(lambda x : str(x).split('/')[-1]).replace('.*united states.*', 'usa', regex = True)

0        philippines
1        philippines
2              india
3              india
4              india
            ...     
59294         poland
59295       colombia
59296           peru
59297           peru
59298         france
Name: customer_country, Length: 59299, dtype: object

In [ ]:
df['customer_country'].fillna('', inplace=True)

In [ ]:
df['country'] = df['customer_country'].apply(lambda x : str(x).split('/')[-1])
df.drop(columns=['customer_country'], inplace= True)

In [ ]:
df['com_reg_ver_win_rate'].fillna(0, inplace=True)

In [ ]:
df['business_unit'].unique()

array(['AS', 'ID', 'IT', 'Solution', 'CM'], dtype=object)

In [ ]:
df.isna().sum()

bant_submit                    0
customer_country               0
business_unit                  0
com_reg_ver_win_rate           0
customer_idx                   0
customer_type              43961
enterprise                     0
historical_existing_cnt    45543
id_strategic_ver               0
it_strategic_ver               0
idit_strategic_ver             0
customer_job               18733
lead_desc_length               0
inquiry_type                 941
product_category           19374
product_subcategory        50064
product_modelname          50070
customer_position              0
response_corporate             0
expected_timeline          30863
ver_cus                        0
ver_pro                        0
ver_win_rate_x             40882
ver_win_ratio_per_bu       43995
business_area              40882
business_subarea           53773
lead_owner                     0
is_converted                   0
dtype: int64

In [ ]:
df['product_category'].nunique()

357

In [ ]:
df['customer_type'] = df['customer_type'].str.lower()

In [ ]:
df[df['customer_type'].str.contains('end') == True]['customer_type'].unique()

array(['end-customer', 'end customer', 'end-user', 'commercial end-user'],
      dtype=object)

In [ ]:
df[df['customer_type'].str.contains('influencer') == True]['customer_type'].unique()

array(['specifier/ influencer', 'specifier / influencer'], dtype=object)

In [ ]:
df[df['customer_type'].str.contains('partner') == True]['customer_type'].unique()

array(['service partner', 'channel partner', 'solution eco-partner'],
      dtype=object)

In [ ]:
df[df['customer_type'].str.contains('engineer') == True]['customer_type'].unique()

array(['engineer', 'hvac engineer'], dtype=object)

In [ ]:
df['customer_type'].str.lower().str.contains('end')

0         True
1         True
2         True
3         True
4        False
         ...  
59294     True
59295    False
59296    False
59297      NaN
59298      NaN
Name: customer_type, Length: 59299, dtype: object

In [ ]:
df['country'] = df['country'].str.lower()
df['country'].value_counts().head(20)

country
india             16578
brazil             8676
united states      3767
                   2980
mexico             2749
philippines        2538
colombia           1963
u.a.e              1617
united kingdom     1487
saudi arabia       1464
chile              1221
italy              1142
peru               1135
germany             894
poland              822
egypt               622
vietnam             597
spain               561
argentina           495
hong kong           391
Name: count, dtype: int64

In [ ]:
df[df['country'].str.contains('united states') == True]['country'].count()

17

In [288]:
test = pd.read_csv('data/submission.csv')